In [1]:
import pandas as pd
from dask.distributed import Client, LocalCluster
import dask.dataframe as dd
from pathlib import Path
import pandas as pd
import os
import numpy as np
from sklearn.neighbors import BallTree
import pyarrow.dataset as ds
import pyarrow as pa
import pyarrow.compute as pc
import shapely
import pandas as pd
from scipy.spatial import cKDTree
import numpy as np


In [ ]:
add_data_path = Path("/d/hpc/home/jv8043/BD/project/T5/add_data")

In [3]:
events_df = pd.read_parquet(add_data_path / "nyc_events_full.parquet")

In [4]:
# count nans in police precint
print("Police precinct nans:", events_df["police_precinct"].isna().sum())

In [5]:
pprecint_loc_df = pd.read_csv(add_data_path / "police_precincts.csv")
pprecint_loc_df.head()

In [6]:
tlc_zones_df = pd.read_csv("/d/hpc/home/jv8043/BD/project/add_data/taxi_zone_lookup.csv")
tlc_zones_df.dropna(subset=["latitude", "longitude"], inplace=True)
tlc_zones_df.head()

In [7]:
import pandas as pd
from shapely.wkt import loads as wkt_loads
from shapely.geometry import Point
import numpy as np
from scipy.spatial import distance

# --- Step 1: Prepare precinct geometries
pprecint_loc_df['geometry'] = pprecint_loc_df['the_geom'].apply(wkt_loads)

# --- Step 2: Get centroids of precincts
pprecint_loc_df['centroid'] = pprecint_loc_df['geometry'].apply(lambda g: g.centroid)
pprecint_loc_df['centroid_lon'] = pprecint_loc_df['centroid'].apply(lambda p: p.x)
pprecint_loc_df['centroid_lat'] = pprecint_loc_df['centroid'].apply(lambda p: p.y)

In [9]:
tlc_zones_df.head()

In [21]:
def find_closest_zone(lat, lon, zones_df):
    distances = zones_df.apply(lambda row: distance.euclidean((lat, lon), (row['latitude'], row['longitude'])), axis=1)
    closest_idx = distances.idxmin()
    return zones_df.loc[closest_idx, ['latitude', 'longitude']]

In [22]:
closest_coords_df = pd.DataFrame(
    pprecint_loc_df.apply(
        lambda row: find_closest_zone(row['centroid_lat'], row['centroid_lon'], tlc_zones_df), axis=1
    ),
    columns=['latitude', 'longitude']
)

# Assign to precincts
pprecint_loc_df[['latitude', 'longitude']] = closest_coords_df

In [25]:
pprecint_loc_df.head()

In [26]:
precinct_coords = pprecint_loc_df[['Precinct', 'latitude', 'longitude']].copy()
precinct_coords['Precinct'] = precinct_coords['Precinct'].astype(int)

events_df['police_precinct'] = events_df['police_precinct'].astype(str).str.extract(r'(\d+)').astype(float)

events_df = events_df.merge(precinct_coords, left_on='police_precinct', right_on='Precinct', how='left')
events_df.drop(columns=['Precinct'], inplace=True)

In [27]:
events_df.head()

In [28]:
events_df = events_df[['event_type', 'start_date_time', 'end_date_time', 'latitude', 'longitude']]
events_df.to_parquet(add_data_path / "nyc_events_augmented.parquet", index=False)